In [1]:
# Import Necessary Libraries
import shapefile
import rasterio
import laspy
import cv2
import math
import numpy as np
from rasterio.control import GroundControlPoint as GCP
from rasterio.transform import from_gcps
import matplotlib.pyplot as plt
import pandas as pd
import whitebox
import os
import numpy as np
from scipy import ndimage
from sklearn.cluster import KMeans
from skimage import io, morphology, measure, color, filters
from sklearn.neighbors import KDTree
from scipy import spatial
import re

In [2]:
# Clear KMeans warning
os.environ['OMP_NUM_THREADS'] = '1'

In [3]:
# CHM Alignment
def transform_chm(chm_path, rtk_points_path):
    with rasterio.open(chm_path) as chm:
        profile = chm.profile
        chm_dir, chm_name = os.path.split(chm_path)

        sf = shapefile.Reader(rtk_points_path, 'rb')
        shapes = sf.shapes()
        rtk_points = [shape.points[0] for shape in shapes]
        # Retrieves the pixel coordinates of CHM from geographic coordinates
        pixel_indices = []
        for x, y in rtk_points:
            row, col = chm.index(x, y)
            pixel_indices.append((row, col))
        print(pixel_indices)

        ul_row, ul_col = pixel_indices[0]
        ll_row, ll_col = pixel_indices[1] 
        lr_row, lr_col = pixel_indices[2]
        ur_row, ur_col = pixel_indices[3]

        num_rows = max(
            np.int64(math.hypot(ul_row - ll_row, ul_col - ll_col)),
            np.int64(math.hypot(ur_row - lr_row, ur_col - lr_col))
        )
        num_cols = max(  
            np.int64(math.hypot(ul_row - ur_row, ul_col - ur_col)),
            np.int64(math.hypot(ll_row - lr_row, ll_col - lr_col))
        )

        print(f"num_rows: {num_rows}, num_cols: {num_cols}")
        print(f"RTK points: {rtk_points}")
        print(f"Pixel indices: {pixel_indices}")
        # make Ground Control Points (GCPs)
        gcps = [
            GCP(0, 0, *rtk_points[0]),
            GCP(0, num_cols, *rtk_points[3]), 
            GCP(num_rows, 0, *rtk_points[1]),
            GCP(num_rows, num_cols, *rtk_points[2])
        ]
        # Generate affine transformation matrix from GCPs
        transform = from_gcps(gcps)
        print(f"Transform: {transform}")

        chm_array = chm.read(1)
        print(f"CHM array shape: {chm_array.shape}")

        src_points = np.float32([
            [ul_col, ul_row], [ur_col, ur_row], 
            [ll_col, ll_row], [lr_col, lr_row]
        ])
        dst_points = np.float32([
            [0, 0], [num_cols, 0],  
            [0, num_rows], [num_cols, num_rows]
        ])
        # Calculate perspective transformation matrix
        perspective_transform = cv2.getPerspectiveTransform(src_points, dst_points)
        print(f"Perspective transform shape: {perspective_transform.shape}")

        # Apply perspective transformation to CHM
        warped_chm = cv2.warpPerspective(chm_array, perspective_transform, (num_cols, num_rows))
        print(f"Warped CHM shape: {warped_chm.shape}")

        profile.update(width=num_cols, height=num_rows, transform=transform)

        transformed_path = os.path.join(chm_dir, "transformed_chm.tif")
        with rasterio.open(transformed_path, 'w', **profile) as dst:
            dst.write(warped_chm, 1)

    return transformed_path

In [4]:
def extract_bbox(image_path):
    # Read the image
    image = io.imread(image_path)

    # Binarization using Otsu's threshold method
    thresh = filters.threshold_otsu(image)
    binary = image > thresh

    # Morphological closing to close canopy areas
    closed = ndimage.binary_closing(
        binary, structure=morphology.disk(5), iterations=6)

    # Remove small isolated noise points
    cleaned = morphology.remove_small_objects(closed, min_size=100)

    # Connected component analysis
    labels = measure.label(cleaned, connectivity=2)

    # Label connected components and display bounding boxes
    rgb_labels = color.label2rgb(labels, bg_label=0)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(binary)

    centroid_list = []
    bbox_list = []
    for region in measure.regionprops(labels):
        # Draw bounding box
        minr, minc, maxr, maxc = region.bbox
        rect = plt.Rectangle((minc, minr), maxc - minc, maxr - minr,
                             fill=False, edgecolor='red', linewidth=1)
        ax.add_patch(rect)
        label = region.label
        ax.text(minc, minr, str(label), fontsize=12, color='yellow',
                ha='center', va='center')

        # Get centroid coordinates and bounding box coordinates
        centroid = region.centroid
        bbox = region.bbox
        centroid_list.append(centroid)
        bbox_list.append(bbox)

    ax.set_axis_off()
    plt.tight_layout()

    # Save result image to IMG folder
    output_dir = os.path.join(os.path.dirname(image_path), 'IMG')
    os.makedirs(output_dir, exist_ok=True)

    # Save the binary image
    binary_path = os.path.join(output_dir, 'binary.png')
    io.imsave(binary_path, binary.astype(np.uint8) * 255)

    # Save the image after closing operation
    closed_path = os.path.join(output_dir, 'closed.png')
    io.imsave(closed_path, closed.astype(np.uint8) * 255)

    # Save the image after removing noise
    cleaned_path = os.path.join(output_dir, 'cleaned.png')
    io.imsave(cleaned_path, cleaned.astype(np.uint8) * 255)

    # Save connected component analysis image
    labels_path = os.path.join(output_dir, 'rgb_labels.png')
    io.imsave(labels_path, (rgb_labels * 255).astype(np.uint8))

    # Save the image with labeled connected components
    result_path = os.path.join(output_dir, 'result.png')
    plt.savefig(result_path, dpi=300, bbox_inches='tight')
    # Generate image of connected component centroids on a white background with black dots

    plt.close(fig)

    return centroid_list, bbox_list,binary

In [5]:
def create_single_tree_shapefile(tree_points_geo, shapefile_path):
    with shapefile.Writer(shapefile_path, shapeType=shapefile.POLYGON) as w:
        w.field('name', 'C')
        w.poly([tree_points_geo])
        w.record('polygon')


# generate trees bouning box
def generate_tree_shapefiles(border_list, chm_path):
    chm = rasterio.open(chm_path)
    geo_pixels = []
    shapefile_paths = []
    col_geo_pixels = []
    for tree_index, tree in enumerate(border_list):
        if len(tree) == 4:
                # Convert the bounding box tuple into a list of coordinate points, and be aware that the direction of the generated .shp file,
                # whether clockwise or counter-clockwise, affects the results of subsequent segmentation. The code is in counter-clockwise direction
                # tree_points = [[tree[0], tree[1]], [tree[2], tree[1]], [tree[2], tree[3]], [tree[0], tree[3]]]
            tree_points = [[tree[0], tree[1]], [tree[0], tree[3]], [tree[2], tree[3]], [tree[2], tree[1]]]
            tree_points_geo = [rasterio.transform.xy(chm.transform, point[0], point[1], offset='center')
                                   for point in tree_points]
            col_geo_pixels.append(tree_points_geo)
            shapefile_name = f"polygon_{tree_index}.shp"
            shapefile_dir = os.path.join(os.path.dirname(chm_path), "Result_dir", "single_tree_shapefiles")
            os.makedirs(shapefile_dir, exist_ok=True)
            shapefile_path = os.path.join(shapefile_dir, shapefile_name)
            shapefile_paths.append(shapefile_path)
            create_single_tree_shapefile(tree_points_geo, shapefile_path)
    geo_pixels.append(col_geo_pixels)
    return geo_pixels, shapefile_paths

def segment_single_tree(las_path, shapefile_paths):
    las_dir, _ = os.path.split(las_path)
    output_dir = os.path.join(las_dir, "Result_dir", "origin_single_tree_las")
    os.makedirs(output_dir, exist_ok=True)

    wbt = whitebox.WhiteboxTools()
    wbt.set_verbose_mode(False)

    for shapefile_path in shapefile_paths:
        # Extract the sequence number from the filename
        file_name = os.path.basename(shapefile_path)
        tree_id = file_name.split('_')[1][:-4]

        output_las_name = f"{tree_id}.las"
        output_las_path = os.path.join(output_dir, output_las_name)

        wbt.clip_lidar_to_polygon(
            i=las_path,
            polygons=shapefile_path,
            output=output_las_path
        )

def denoise_pear_tree(input_folder, output_folder, sigma=1, K=10):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Iterate over all LAS files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".las") or filename.endswith(".LAS"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            las = laspy.read(input_path)
            x, y, z = las["X"], las["Y"], las["Z"]
            lasdata = (np.stack((x, y, z), axis=0)).transpose()
            tree = spatial.KDTree(lasdata)

            k_dist = np.zeros_like(x)
            for i in range(len(x)):
                dist, index = tree.query(np.array([x[i], y[i], z[i]]), K)
                k_dist[i] = np.sum(dist)

            # Determine the maximum threshold of noise
            max_distance = np.mean(k_dist) + sigma * np.std(k_dist)

            # Index of noise
            outer_index = np.where(k_dist > max_distance)
            # print(f'Outer points index array for {filename}: {outer_index}')

            sor_filter = k_dist <= max_distance

            out_las = laspy.LasData(las.header)
            out_las.points = las.points[sor_filter]
            out_las.write(output_path)

# Lidar Dormancy Stage

In [9]:
lidar_chm_path = r"E:\Pear\Lidar\Dormancy Stage\chm_roi_lidar_dormancy.tif"
lidar_rtk_path = r"E:\Pear\ROI_Points\Lidar\lidar_roi_points.shp"
lidar_transformed_chm_path = transform_chm(lidar_chm_path, lidar_rtk_path)

[(-1, 2278), (2626, -1), (4941, 2629), (2290, 4976)]
num_rows: 3540, num_cols: 3539
RTK points: [[-2185230.64999, 13857645.31], [-2185253.44, 13857619.04], [-2185227.14, 13857595.89], [-2185203.67, 13857622.4]]
Pixel indices: [(-1, 2278), (2626, -1), (4941, 2629), (2290, 4976)]
Transform: | 0.01,-0.01,-2185230.48|
|-0.01,-0.01, 13857645.37|
| 0.00, 0.00, 1.00|
CHM array shape: (4941, 4975)
Perspective transform shape: (3, 3)
Warped CHM shape: (3540, 3539)


In [10]:
# BBox Detection
lidar_centroid_list, lidar_bbox_list, lidar_binary = extract_bbox(lidar_transformed_chm_path)
print("Centroid coordinates:")
print(lidar_centroid_list)
print("Bounding box coordinates:")
print(lidar_bbox_list)

Centroid coordinates:
[(166.91353459288644, 264.06928662677336), (184.59540043489477, 1430.0260199756754), (176.29453345900095, 2601.0295711592835), (164.69394313967862, 2945.1448207663784), (170.46156176954375, 2204.019435546706), (161.34891242310863, 1058.6161456021307), (204.2296951739254, 3363.9536063644114), (160.33614435418863, 1818.9319774291562), (608.1146100329549, 255.562504577078), (607.0545156645649, 3352.4277841219405), (579.4148780906594, 1058.915307348901), (615.8156586809628, 1813.2812308035964), (621.0580650393317, 2593.2300705538887), (633.5559529344443, 2991.563677980044), (606.9983529036714, 2209.8692418043674), (605.6870558544199, 660.9739655973966), (585.2739726027397, 1431.7769582016158), (1025.023860247124, 1441.5856838517257), (1045.974631582056, 255.47895809338178), (1061.4162874732333, 2962.927663276231), (1003.47768896721, 1051.6766945685695), (1053.7003623641135, 2209.224040984631), (1062.2863740512673, 2601.6634684233136), (1062.604708846805, 3343.14965156

In [11]:
lidar_geo_pixels, lidar_shapefile_paths = generate_tree_shapefiles(lidar_bbox_list,lidar_transformed_chm_path)

In [12]:
# segment uav flowering stage single tree
lidar_las_path = r"E:\Pear\Lidar\Dormancy Stage\dormancy_manual_above_ground_sor_roi_lidar.las"
segment_single_tree(lidar_las_path, lidar_shapefile_paths)

In [13]:
lidar_single_tree_folder = r"E:\Pear\Lidar\Dormancy Stage\Result_dir\origin_single_tree_las"
lidar_denosied_tree_folder = r"E:\Pear\Lidar\Dormancy Stage\Result_dir\denosied_single_tree_las"
denoise_pear_tree(lidar_single_tree_folder, lidar_denosied_tree_folder, sigma=5, K=10)

# Uav Dormancy Stage

In [14]:
uav_dormancy_chm_path = r"E:\Pear\UAV\Dormancy Stage\chm_roi_uav_dormancy.tif"
uav_dormancy_rtk_path = r"E:\Pear\ROI_Points\UAV\uav_roi_points.shp"
uav_dormancy_transformed_chm_path = transform_chm(uav_dormancy_chm_path, uav_dormancy_rtk_path)

[(-1, 1402), (2925, -1), (4324, 2936), (1373, 4373)]
num_rows: 3282, num_cols: 3273
RTK points: [[421290.74092155846, 3499820.730711485], [421276.7025521803, 3499791.4654444326], [421306.0758895532, 3499777.4790258044], [421320.4454896842, 3499806.991575519]]
Pixel indices: [(-1, 1402), (2925, -1), (4324, 2936), (1373, 4373)]
Transform: | 0.01,-0.00, 421290.82|
|-0.00,-0.01, 3499820.79|
| 0.00, 0.00, 1.00|
CHM array shape: (4323, 4373)
Perspective transform shape: (3, 3)
Warped CHM shape: (3282, 3273)


In [15]:
# BBox Detection
uav_dormancy_centroid_list, uav_dormancy_bbox_list, uav_dormancy_binary = extract_bbox(uav_dormancy_transformed_chm_path)

print("Centroid coordinates:")
print(uav_dormancy_centroid_list)
print("Bounding box coordinates:")
print(uav_dormancy_bbox_list)

Centroid coordinates:
[(227.29531001589825, 2403.3428722840486), (207.736400541272, 260.51132160577356), (241.1831606788627, 2032.5369186687237), (229.79100664282063, 2729.817705671947), (214.55038520801233, 1687.3554699537751), (241.68477366255144, 1326.2041152263375), (267.3015442969385, 3105.4181793551884), (221.1148798707854, 985.4017767009893), (606.0501074428599, 231.9431529595624), (640.7148859543818, 3093.596138455382), (661.4063621143267, 2763.6430678466077), (623.6922833148471, 2037.7111392806212), (657.4356969830434, 2391.633560889672), (595.6990445859873, 989.3097857556456), (608.0520757465404, 616.8170065549891), (625.6539205155747, 1665.2554242749732), (602.4071601941747, 1326.9726941747572), (1015.8554275515171, 243.41732922278112), (997.5668321067944, 1322.739003936334), (989.3536282306163, 976.2365805168986), (1032.886013986014, 2050.279160839161), (1046.8820203255138, 2728.564911744479), (1026.2563688554, 1655.066917832795), (1060.783828816978, 3080.395860738402), (10

In [16]:
uav_dormancy_geo_pixels, uav_dormancy_shapefile_paths = generate_tree_shapefiles(uav_dormancy_bbox_list, uav_dormancy_transformed_chm_path)

In [17]:
# segment uav flowering stage single tree
uav_dormancy_las_path = r"E:\Pear\UAV\Dormancy Stage\dormancy_manual_above_ground_sor_roi_uav.las"
segment_single_tree(uav_dormancy_las_path, uav_dormancy_shapefile_paths)

In [18]:
uav_dormancy_single_tree_folder = r"E:\Pear\UAV\Dormancy Stage\Result_dir\origin_single_tree_las"
uav_dormancy_denosied_tree_folder = r"E:\Pear\UAV\Dormancy Stage\Result_dir\denosied_single_tree_las"
denoise_pear_tree(uav_dormancy_single_tree_folder, uav_dormancy_denosied_tree_folder, sigma=1, K=10)

# Uav Flowering Stage

In [20]:
uav_flowering_chm_path = r"E:\Pear\UAV\Flowering Stage\chm_roi_uav_flowering.tif"
uav_flowering_rtk_path = r"E:\Pear\ROI_Points\UAV\uav_roi_points.shp"
uav_flowering_transformed_chm_path = transform_chm(uav_flowering_chm_path, uav_flowering_rtk_path)

[(-18, 1403), (2908, -1), (4307, 2936), (1355, 4373)]
num_rows: 3283, num_cols: 3272
RTK points: [[421290.74092155846, 3499820.730711485], [421276.7025521803, 3499791.4654444326], [421306.0758895532, 3499777.4790258044], [421320.4454896842, 3499806.991575519]]
Pixel indices: [(-18, 1403), (2908, -1), (4307, 2936), (1355, 4373)]
Transform: | 0.01,-0.00, 421290.82|
|-0.00,-0.01, 3499820.79|
| 0.00, 0.00, 1.00|
CHM array shape: (4307, 4366)
Perspective transform shape: (3, 3)
Warped CHM shape: (3283, 3272)


In [21]:
# BBox Detection
uav_flowering_centroid_list, uav_flowering_bbox_list, uav_flowering_binary = extract_bbox(uav_flowering_transformed_chm_path)

print("Centroid coordinates:")
print(uav_flowering_centroid_list)
print("Bounding box coordinates:")
print(uav_flowering_bbox_list)

Centroid coordinates:
[(198.78633838762948, 242.7431616874343), (229.76985329293575, 1312.4612501993302), (235.27661567280674, 2392.551306067229), (222.03123340574464, 2706.832923002655), (227.53209375517642, 2027.2459002815967), (200.59889582333173, 973.1279404704753), (209.11273449827596, 1671.077786219391), (265.1341603053435, 3095.8957251908396), (609.7934133790737, 234.73186963979416), (590.4227290223249, 977.7211797536567), (623.554576305526, 1661.0107886339463), (635.8431641714212, 3080.6564711938568), (641.6203768896252, 2383.3483122799753), (651.7759520587666, 2752.8011791996905), (628.0757741002418, 2025.4301294513727), (612.1622143991165, 608.5825222613378), (594.7412828239345, 1315.4089539388722), (1013.3658556192722, 230.95897422155528), (1004.1341363150539, 1316.4235797089534), (976.7328200741374, 962.9211861990306), (1052.1237133808393, 2718.9690881499077), (1039.2757189554486, 2026.8249164432366), (1056.7650760774318, 3070.1718314347168), (1056.1542711432214, 2383.09294

In [22]:
uav_flowering_geo_pixels, uav_flowering_shapefile_paths = generate_tree_shapefiles(uav_flowering_bbox_list, uav_flowering_transformed_chm_path)

In [23]:
# segment uav flowering stage single tree
uav_flowering_las_path = r"E:\Pear\UAV\Flowering Stage\flowering_manual_above_ground_sor_roi_uav.las"
segment_single_tree(uav_flowering_las_path, uav_flowering_shapefile_paths)

In [24]:
uav_flowering_single_tree_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir\origin_single_tree_las"
uav_flowering_denosied_tree_folder = r"E:\Pear\UAV\Flowering Stage\Result_dir\denosied_single_tree_las"
denoise_pear_tree(uav_flowering_single_tree_folder, uav_flowering_denosied_tree_folder, sigma=1, K=10)